<a href="https://colab.research.google.com/github/jhuarancca/Data-Science-for-Healthcare-Claims-data/blob/main/Assigment03_Logistic_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Import the relevant libraries**

In [40]:
import pandas as pd
import numpy as np
import io  # If you are not using Google Colab, skip the last sentence containing 'import io'
import statsmodels.api as sm # this library has now been added because it is needed for logistic regression 


**Upload the dataset**

**Upload the alternative category look up table**

**Import the two datasets**

In [41]:
raw_data1 = pd.read_csv('OBGYN.csv')
OBGYN_category_v2 = pd.read_csv('OBGYN_category_v2.csv')

# If you are not using Google Colab, use a different code:
# raw_data = pd.read_csv(r'C:\Users\location of the csv file on your computer\\OBGYN_with categories.csv')


**Merge the two dataframes in order to get the alternative categorization**

In [42]:
merged = pd.merge(raw_data1, OBGYN_category_v2, on="procedure_concept_id")


**Inspect the dataset**

In [43]:
merged

,person_id,procedure_date,procedure_concept_id,concept_name_x,amount,total_paid,provider_id,provider_name,category according to domain expert,category according to process mining,concept_name_y
0,11248,8/3/2020,2004407,Amputation of cervix,1,839,45192,Baptist Health & Medical Center,Procedure,Gynaecology,Amputation of cervix
1,11874,4/6/2020,2004407,Amputation of cervix,1,839,52356,Boudica Female Care,Procedure,Gynaecology,Amputation of cervix
2,12745,12/20/2019,2004407,Amputation of cervix,1,839,24023,dr. Mariano & Co.,Procedure,Gynaecology,Amputation of cervix
3,13133,7/29/2019,2004407,Amputation of cervix,1,839,24023,dr. Mariano & Co.,Procedure,Gynaecology,Amputation of cervix
4,13844,8/31/2020,2004407,Amputation of cervix,1,839,45192,Baptist Health & Medical Center,Procedure,Gynaecology,Amputation of cervix
...,...,...,...,...,...,...,...,...,...,...,...
52188,19298,9/26/2019,45890386,Abortion Procedures,1,1129,45192,Baptist Health & Medical Center,Procedure,Gynaecology,Abortion Procedures
52189,19406,1/18/2020,45890386,Abortion Procedures,1,1129,45192,Baptist Health & Medical Center,Procedure,Gynaecology,Abortion Procedures
52190,19435,8/26/2020,45890386,Abortion Procedures,1,1129,4823,East Westfield Health Services,Procedure,Gynaecology,Abortion Procedures
52191,19656,4/30/2020,45890386,Abortion Procedures,1,1129,45192,Baptist Health & Medical Center,Procedure,Gynaecology,Abortion Procedures


**Create a pivot table**

In [44]:
pivot = pd.pivot_table(merged, values = 'amount', index=['person_id'], columns = 'category according to process mining',aggfunc=np.sum).reset_index()


**Inspect the pivot table**

In [45]:
pivot

category according to process mining,person_id,Consultation,Delivery,Gynaecology,Obstetrics
0,10429,4.0,NaN,3.0,NaN
1,10430,10.0,1.0,NaN,8.0
2,10431,9.0,1.0,NaN,7.0
3,10432,6.0,NaN,NaN,6.0
4,10433,1.0,NaN,NaN,NaN
...,...,...,...,...,...
9258,19688,NaN,1.0,NaN,10.0
9259,19689,2.0,NaN,2.0,NaN
9260,19690,10.0,1.0,NaN,8.0
9261,19691,1.0,NaN,NaN,NaN


**Replace blank cells with zero**

In [46]:
raw_data = pivot.fillna(0)

**Inspect the new dataset**

In [47]:
raw_data

category according to process mining,person_id,Consultation,Delivery,Gynaecology,Obstetrics
0,10429,4.0,0.0,3.0,0.0
1,10430,10.0,1.0,0.0,8.0
2,10431,9.0,1.0,0.0,7.0
3,10432,6.0,0.0,0.0,6.0
4,10433,1.0,0.0,0.0,0.0
...,...,...,...,...,...
9258,19688,0.0,1.0,0.0,10.0
9259,19689,2.0,0.0,2.0,0.0
9260,19690,10.0,1.0,0.0,8.0
9261,19691,1.0,0.0,0.0,0.0


**Retrieve the names of the columns**

In [48]:
raw_data.columns

Index(['person_id', 'Consultation', 'Delivery', 'Gynaecology', 'Obstetrics'], dtype='object', name='category according to process mining')

**Rearrange the columns to place the target variable at the end**

In [49]:
dataset = raw_data[['person_id', 'Consultation',  'Gynaecology', 'Obstetrics','Delivery']]

**Inspect the new dataset**

In [50]:
dataset

category according to process mining,person_id,Consultation,Gynaecology,Obstetrics,Delivery
0,10429,4.0,3.0,0.0,0.0
1,10430,10.0,0.0,8.0,1.0
2,10431,9.0,0.0,7.0,1.0
3,10432,6.0,0.0,6.0,0.0
4,10433,1.0,0.0,0.0,0.0
...,...,...,...,...,...
9258,19688,0.0,0.0,10.0,1.0
9259,19689,2.0,2.0,0.0,0.0
9260,19690,10.0,0.0,8.0,1.0
9261,19691,1.0,0.0,0.0,0.0


**Inspect if the target is binary**

In [51]:
dataset.Delivery.value_counts()

0.0    8110
1.0    1143
2.0      10
Name: Delivery, dtype: int64

**Convert the target to binary values**

In [52]:
Data_with_binary_targets = dataset.copy()
Binary = Data_with_binary_targets['Delivery_binary'] = np.where(Data_with_binary_targets['Delivery'] > 0, 1, 0)
Data_with_binary_targets['Delivery_binary'] = Binary
Data_with_binary_targets

category according to process mining,person_id,Consultation,Gynaecology,Obstetrics,Delivery,Delivery_binary
0,10429,4.0,3.0,0.0,0.0,0
1,10430,10.0,0.0,8.0,1.0,1
2,10431,9.0,0.0,7.0,1.0,1
3,10432,6.0,0.0,6.0,0.0,0
4,10433,1.0,0.0,0.0,0.0,0
...,...,...,...,...,...,...
9258,19688,0.0,0.0,10.0,1.0,1
9259,19689,2.0,2.0,0.0,0.0,0
9260,19690,10.0,0.0,8.0,1.0,1
9261,19691,1.0,0.0,0.0,0.0,0


**Confirm the binary values by counting the values in the target column**

In [53]:
Data_with_binary_targets.Delivery_binary.value_counts()

0    8110
1    1153
Name: Delivery_binary, dtype: int64

**Remove the column 'Delivery' that is non-binary**

In [54]:
Dataframe = Data_with_binary_targets.drop(columns=['Delivery'])

**Assign the predictive variables (x1) and the target variable (y)**

In [55]:
x1 = Dataframe.iloc[:,1:-1]
y = Dataframe.iloc[:,-1]


**Confirm x1**

In [56]:
x1

category according to process mining,Consultation,Gynaecology,Obstetrics
0,4.0,3.0,0.0
1,10.0,0.0,8.0
2,9.0,0.0,7.0
3,6.0,0.0,6.0
4,1.0,0.0,0.0
...,...,...,...
9258,0.0,0.0,10.0
9259,2.0,2.0,0.0
9260,10.0,0.0,8.0
9261,1.0,0.0,0.0


**Confirm y**

In [57]:
y

0       0
1       1
2       1
3       0
4       0
       ..
9258    1
9259    0
9260    1
9261    0
9262    0
Name: Delivery_binary, Length: 9263, dtype: int64

**Add a constant** (For more info, see https://www.statsmodels.org/devel/generated/statsmodels.regression.linear_model.OLS.html )

In [58]:
x = sm.add_constant(x1)

/usr/local/lib/python3.8/dist-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


**Apply logistic regression to the data**

In [59]:
reg_log = sm.Logit(y,x)
results_log = reg_log.fit()

Optimization terminated successfully.
         Current function value: 0.166984
         Iterations 8


**Inspect the summary of the logistic regression**

In [60]:
results_log.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:        Delivery_binary   No. Observations:                 9263
Model:                          Logit   Df Residuals:                     9259
Method:                           MLE   Df Model:                            3
Date:                Wed, 15 Feb 2023   Pseudo R-squ.:                  0.5556
Time:                        14:53:35   Log-Likelihood:                -1546.8
converged:                       True   LL-Null:                       -3480.5
Covariance Type:            nonrobust   LLR p-value:                     0.000
================================================================================
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
const           -3.2369      0.081    -39.811      0.000      -3.396      -3.078
Consultation     0.0278      0.017      1.614      0.107      -0.006       0.061
Gynaecology     -0.7065      0.067    -10.552      0.000      -0.838      -0.575
Obstetrics       0.6712      0.021     31.271      0.000       0.629       0.713
================================================================================
"""

**Evaluate the results with a confusion matrix**

In [61]:
cm = results_log.pred_table()
cm_df = pd.DataFrame(cm)
cm_df.columns = ['Predicted 0', 'Predicted 1']
cm_df = cm_df.rename(index={0:'Actual 0', 1:'Actual 1'})
cm_df  

,Predicted 0,Predicted 1
Actual 0,7909.0,201.0
Actual 1,398.0,755.0


**Calculate the accuracy of the model**

In [62]:
(cm[0,0]+cm[1,1])/np.sum([cm])  

0.9353341250134946